# Verificação de Ambiente de GPU

Este notebook serve como um script de diagnóstico para verificar a configuração do ambiente de computação acelerada por GPU. Ele examina a disponibilidade e a configuração do TensorFlow, PyTorch e dos drivers NVIDIA.

As verificações realizadas incluem:
1.  **Versões de Python e Frameworks**: Exibe as versões instaladas do Python, TensorFlow e PyTorch.
2.  **Detecção de GPU pelo TensorFlow**: Verifica se o TensorFlow consegue detectar e listar as GPUs físicas disponíveis e com qual versão do CUDA/cuDNN foi construído.
3.  **Detecção de GPU pelo PyTorch**: Confirma se o PyTorch pode acessar o CUDA, lista as GPUs visíveis e mostra suas propriedades, como nome e memória.
4.  **Status do Driver NVIDIA**: Executa o comando `nvidia-smi` para obter informações diretamente do driver da NVIDIA, incluindo o nome da GPU, a versão do driver e o uso atual.

O objetivo é fornecer um relatório abrangente que ajude a diagnosticar rapidamente problemas comuns de configuração, como drivers incompatíveis, instalações incorretas de frameworks ou problemas de visibilidade da GPU.

In [1]:
import tensorflow as tf
import sys
import subprocess
import json
import torch
import textwrap
from datetime import datetime

### Verificação da Versão do Python

Exibe a versão do interpretador Python que está executando o notebook.

In [2]:
print("Python:", sys.version.splitlines()[0])
print()

### Verificação do TensorFlow e CUDA/cuDNN

Esta célula verifica a instalação do TensorFlow. Ela tenta importar a biblioteca, imprime a versão e, o mais importante, verifica se o TensorFlow consegue detectar as GPUs físicas. Também exibe as versões do CUDA e do cuDNN com as quais o TensorFlow foi construído, o que é crucial para a compatibilidade.

In [3]:
try:
    import tensorflow as tf
    print("TensorFlow:", tf.__version__)
    gpus = tf.config.list_physical_devices('GPU')
    print("  GPUs detectadas por TensorFlow:", len(gpus))
    for i, g in enumerate(gpus):
        print(f"    [{i}] {g}")
    try:
        print("  TF built with CUDA:", tf.test.is_built_with_cuda())
    except Exception:
        # fallback: algumas versões não têm is_built_with_cuda
        print("  TF built with CUDA: (não disponível nesta versão do TF)")
except Exception as e:
    print("TensorFlow: não disponível ou erro ao importar:", e)

print()


build = tf.sysconfig.get_build_info()
print("Versão CUDA:", build.get("cuda_version", "desconhecida"))
print("Versão cuDNN:", build.get("cudnn_version", "desconhecida"))

### Verificação do PyTorch e CUDA

Semelhante à verificação do TensorFlow, esta célula verifica a instalação do PyTorch. Ela confirma se o PyTorch foi instalado com suporte a CUDA (`torch.cuda.is_available()`), conta o número de GPUs detectadas e lista o nome e a memória de cada uma. A versão do CUDA com a qual o PyTorch foi compilado também é exibida.

In [4]:
try:
    import torch
    print("PyTorch:", torch.__version__)
    print("  torch.cuda.is_available():", torch.cuda.is_available())
    try:
        count = torch.cuda.device_count()
        print("  GPUs detectadas por PyTorch:", count)
        for i in range(count):
            print(f"    [{i}] {torch.cuda.get_device_name(i)}")
            mem = torch.cuda.get_device_properties(i).total_memory
            print(f"      Memória total (bytes): {mem}")
    except Exception as e:
        print("  Erro ao obter detalhes da GPU via PyTorch:", e)
    print("  CUDA (compilado):", torch.version.cuda)
except Exception as e:
    print("PyTorch: não disponível ou erro ao importar:", e)

print()


### Verificação do Driver NVIDIA com `nvidia-smi`

Esta célula interage diretamente com o sistema operacional para executar o comando `nvidia-smi`, a ferramenta de linha de comando da NVIDIA. Isso fornece uma "fonte da verdade" sobre o que o driver da NVIDIA está relatando, incluindo as GPUs detectadas, a versão do driver e a memória total. É uma forma independente de verificar a configuração do hardware, sem depender dos frameworks de aprendizado de máquina.

In [5]:
def run_cmd(cmd: str) -> str | None:
    """Executa um comando de shell e retorna sua saída como uma string.

    Args:
        cmd (str): O comando a ser executado.

    Returns:
        str | None: A saída do comando decodificada como UTF-8 ou None se o comando falhar.
    """
    try:
        out = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
        return out.decode('utf-8', errors='ignore')
    except subprocess.CalledProcessError as e:
        return None

nvsmi = run_cmd("nvidia-smi -L")
if nvsmi:
    print("nvidia-smi -L (GPUs detectadas):")
    print(nvsmi.strip())
    print("nvidia-smi (sumário):")
    print(run_cmd("nvidia-smi --query-gpu=name,index,memory.total,driver_version --format=csv"))
else:
    print("nvidia-smi não encontrado ou não respondeu. Pode não haver driver NVIDIA instalado ou o utilitário não está no PATH.")
    # tentar informações do kernel
    mod = run_cmd("lsmod | grep nvidia || true")
    print("Módulos nvidia carregados (lsmod | grep nvidia):")
    print(mod or "(nenhum)")

### Exemplo de Uso de Dispositivo em PyTorch

Demonstra a prática padrão no PyTorch para selecionar a GPU (se disponível) ou recorrer à CPU. O objeto `device` é então usado para mover tensores e modelos para o hardware apropriado durante o treinamento ou inferência.

In [6]:
import torch

# cria device explicitamente (assumindo que cuda está disponível)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Usando device:", device)


### Relatório de Diagnóstico Completo

Esta célula final combina todas as verificações anteriores em um único script de diagnóstico. Ele gera um relatório claro e formatado com informações do Python, TensorFlow, PyTorch e `nvidia-smi`. Além disso, inclui dicas de solução de problemas para ajudar a interpretar os resultados e corrigir problemas comuns de configuração de GPU.

In [ ]:
# Célula: verificação completa de GPU (TensorFlow, PyTorch, nvidia-smi)
def hr(title: str = "") -> None:
    """Imprime uma linha horizontal com um título opcional para formatar a saída.

    Args:
        title (str, optional): O título a ser exibido acima da linha. O padrão é "".
    """
    print("\n" + "="*80)
    if title:
        print(title)
        print("-"*len(title))

def human_bytes(n: int) -> str:
    """Converte um número de bytes em um formato legível por humanos (KB, MB, GB).

    Args:
        n (int): O número de bytes.

    Returns:
        str: A representação da string formatada.
    """
    for unit in ['B','KB','MB','GB','TB']:
        if abs(n) < 1024.0:
            return f"{n:3.1f}{unit}"
        n /= 1024.0
    return f"{n:.1f}PB"

print(f"Relatório gerado em: {datetime.now().isoformat(sep=' ', timespec='seconds')}")
hr("Python / Ambiente")
print("Python:", sys.version.splitlines()[0])

# TensorFlow
hr("TensorFlow")
try:
    import tensorflow as tf
    print("tensorflow:", tf.__version__)
    try:
        gpus = tf.config.list_physical_devices('GPU')
        print("  GPUs detectadas por TensorFlow:", len(gpus))
        for i, g in enumerate(gpus):
            print(f"    [{i}] {g}")
    except Exception as e:
        print("  Erro ao listar GPUs no TF:", e)
    try:
        built_cuda = tf.test.is_built_with_cuda()
        print("  TF built with CUDA:", built_cuda)
    except Exception:
        # fallback se função não existir
        build = tf.sysconfig.get_build_info()
        print("  TF build info (cuda/cudnn):",
              f"cuda={build.get('cuda_version','?')}, cudnn={build.get('cudnn_version','?')}")
except Exception as e:
    print("TensorFlow não disponível / erro ao importar:", str(e))

# PyTorch
hr("PyTorch")
try:
    import torch
    print("torch:", torch.__version__)
    cuda_avail = torch.cuda.is_available()
    print("  torch.cuda.is_available():", cuda_avail)
    try:
        dev_count = torch.cuda.device_count()
        print("  GPUs detectadas por PyTorch:", dev_count)
        for i in range(dev_count):
            try:
                name = torch.cuda.get_device_name(i)
            except Exception:
                name = "(nome indisponível)"
            try:
                props = torch.cuda.get_device_properties(i)
                mem = props.total_memory
            except Exception:
                mem = None
            print(f"    [{i}] {name}")
            if mem:
                print(f"       Memória total: {human_bytes(mem)}")
    except Exception as e:
        print("  Erro ao obter detalhes via PyTorch:", e)
    try:
        print("  CUDA (compilado):", torch.version.cuda)
    except Exception:
        pass
except Exception as e:
    print("PyTorch não disponível / erro ao importar:", str(e))

# nvidia-smi / driver
hr("nvidia-smi / Driver")
nvsmi_path = run_cmd("which nvidia-smi")
if nvsmi_path:
    print("nvidia-smi encontrado em:", nvsmi_path)
    print("\nSaída: nvidia-smi -L")
    print(run_cmd("nvidia-smi -L") or "(vazio)")
    print("\nSumário (name,index,memory.total,driver_version):")
    print(run_cmd("nvidia-smi --query-gpu=name,index,memory.total,driver_version --format=csv") or "(não retornou)")
    print("\nUtilização / status (top):")
    print(run_cmd("nvidia-smi --query-gpu=index,name,temperature.gpu,utilization.gpu,utilization.memory --format=csv,nounits") or "(não retornou)")
else:
    print("nvidia-smi não encontrado no PATH. Pode não haver driver NVIDIA instalado ou utilitário não está disponível.")
    # tenta checar módulos do kernel
    mod = run_cmd("lsmod | grep -i nvidia || true")
    print("\nMódulos nvidia carregados (lsmod | grep -i nvidia):")
    print(mod or "(nenhum)")

# Dicas rápidas
hr("Dicas rápidas / interpretação")
print(textwrap.dedent("""
- Se 'nvidia-smi' não existir: driver NVIDIA ausente; instale driver apropriado para sua distribuição.
- Se 'nvidia-smi' mostrar GPUs, mas torch.cuda.is_available() for False:
  - possivelmente PyTorch foi instalado sem suporte CUDA ou há incompatibilidade entre driver e runtime CUDA.
  - solução rápida: usar conda e instalar pytorch + pytorch-cuda compatível com sua versão de driver.
- Se TensorFlow não vê GPU: verifique se instalou a versão GPU (ou compatível) do TensorFlow e a compatibilidade CUDA/cuDNN.
- Em Docker: rode com '--gpus all' e instale 'nvidia-docker2' / 'nvidia-container-toolkit'.
- Se quiser, cole a saída completa desta célula aqui que eu indico os comandos exatos para instalar drivers / bibliotecas.
"""))
hr("Fim do relatório")